In [1]:
!git clone https://github.com/elenaajayi/learning-addition-transformer.git
%cd learning-addition-transformer
!ls -la *.py

Cloning into 'learning-addition-transformer'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 2), reused 14 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 24.70 KiB | 8.23 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/learning-addition-transformer
-rw-r--r-- 1 root root  5115 Jan  1 16:33 config.py
-rw-r--r-- 1 root root 17808 Jan  1 16:33 data_generator.py
-rw-r--r-- 1 root root 17359 Jan  1 16:33 evaluate.py
-rw-r--r-- 1 root root 19001 Jan  1 16:33 model.py
-rw-r--r-- 1 root root 19711 Jan  1 16:33 train.py


In [2]:
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

GPU available: True
GPU: Tesla T4


In [3]:
!pip install -q -r requirements.txt

In [4]:
!python data_generator.py
!ls -lh data/

GENERATING ADDITION DATASETS

Configuration:
  k_digits: 3
  Number range: [100, 999]

[Generating training data...]
[Generating validation data...]
[Generating test data...]

[Saving datasets...]
Saved 50000 examples to data/train.json
Saved 5000 examples to data/val.json
Saved 10000 examples to data/test.json

----------------------------------------
DATASET STATISTICS
----------------------------------------

Train set:
  Total examples: 50,000
  With carry: 43,271 (86.5%)
  Without carry: 6,729 (13.5%)

Val set:
  Total examples: 5,000
  With carry: 4,311 (86.2%)
  Without carry: 689 (13.8%)

Test set:
  Total examples: 10,000
  With carry: 8,633 (86.3%)
  Without carry: 1,367 (13.7%)


[Generating length generalization tests...]
Saved 1000 examples to data/test_length_k4.json
Saved 1000 examples to data/test_length_k5.json

[Generating distribution shift tests...]
Saved 378 examples to data/test_shift_many_nines.json
Saved 1000 examples to data/test_shift_small_numbers.json

Done!

In [5]:
!python config.py

Testing...
CONFIGURATION

[TASK CONFIG]
  K_DIGITS: 3
  TOKENIZATION: character
  VOCAB_SIZE: 15
  Vocabulary: ['PAD', 'BOS', 'EOS', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '=']

[DATA CONFIG]
  TRAIN_SIZE: 50,000
  VAL_SIZE: 5,000
  TEST_SIZE: 10,000
  SAMPLING_STRATEGY: uniform
  SEED: 42
  DATA_DIR: data

[MODEL CONFIG]
  N_LAYERS: 6
  D_MODEL: 256
  N_HEADS: 8
  D_FF: 1024
  DROPOUT: 0.1
  MAX_SEQ_LEN: 32
  VOCAB_SIZE: 15

[TRAIN CONFIG]
  BATCH_SIZE: 128
  LEARNING_RATE: 0.0001
  WEIGHT_DECAY: 0.01
  MAX_EPOCHS: 50
  PATIENCE: 5
  DEVICE: cuda
  SAVE_DIR: checkpoints
  SEED: 42

[EVAL CONFIG]
  EVAL_STRATEGIES: ['uniform', 'carry', 'no_carry', 'edge_cases']
  TEST_SIZE_PER_STRATEGY: 1000
  DECODING: greedy
  METRICS: ['accuracy', 'exact_match', 'digit_accuracy']
  RESULTS_FILE: results.json

Test complete.


In [6]:
# Train!
import time
start = time.time()
!python train.py
print(f"Done in {(time.time()-start)/60:.1f} min")

ADDITION TRANSFORMER TRAINING

[Setup]
  Device: cuda
  Random seed: 42
  Vocabulary size: 15

[Data]
  Found existing datasets in data/
Loaded 50000 examples from data/train.json
Loaded 5000 examples from data/val.json
  Training batches: 391
  Validation batches: 40

[Model]
  Parameters: 4,746,767
  Architecture: 6 layers, d_model=256

[Training Setup]
  Optimizer: AdamW (lr=0.0001, wd=0.01)
  Scheduler: CosineAnnealingLR (T_max=50)
  Loss: CrossEntropyLoss (ignore_index=0)

[Training]
  Max epochs: 50
  Early stopping patience: 5
------------------------------------------------------------

Epoch 1/50
  Train Loss: 1.8184
  Val Loss: 1.6720 | Val Acc: 0.00% | Answer Acc: 0.00%
  Learning Rate: 1.00e-04
  ✓ Saved best model (val_loss: 1.6720)

Epoch 2/50
  Train Loss: 1.6725
  Val Loss: 1.6156 | Val Acc: 0.00% | Answer Acc: 0.00%
  Learning Rate: 9.99e-05
  ✓ Saved best model (val_loss: 1.6156)

Epoch 3/50
  Train Loss: 1.6083
  Val Loss: 1.5495 | Val Acc: 0.00% | Answer Acc: 0.20%


In [8]:
#Evaluate
!python evaluate.py

Loaded model from checkpoints/best_model.pt on cuda

RUNNING FULL EVALUATION


Evaluating: In-Distribution (k=3)
In-Distribution (k=3): 100% 10000/10000 [03:34<00:00, 46.55it/s]

Evaluating: Length Gen (k=4)
Length Gen (k=4): 100% 1000/1000 [00:10<00:00, 91.07it/s]

Evaluating: Length Gen (k=5)
Length Gen (k=5): 100% 1000/1000 [00:04<00:00, 211.93it/s]

Evaluating: Dist Shift (Many 9s)
Dist Shift (Many 9s): 100% 378/378 [00:08<00:00, 43.18it/s]

Evaluating: Dist Shift (Small Nums)
Dist Shift (Small Nums): 100% 1000/1000 [00:18<00:00, 53.66it/s]


┌────────────────────────────┬────────┬──────────┬────────────┬────────────┐
│Test Set                    │ Total  │ Accuracy │ Carry Acc  │  No-Carry  │
├────────────────────────────┼────────┼──────────┼────────────┼────────────┤
│In-Distribution (k=3)       │ 10000  │  94.8%   │   94.3%    │   98.0%    │
│Length Gen (k=4)            │  1000  │   0.0%   │    0.0%    │    0.0%    │
│Length Gen (k=5)            │  1000  │   0.0%   │    0.0%    

In [18]:
# Load and display results
import json

with open('results/evaluation_results.json', 'r') as f:
    results = json.load(f)

print("=" * 60)
print("BASELINE EXPERIMENT (k=3 only) RESULTS")
print("=" * 60)

for test_name, metrics in results['results'].items():
    print(f"\n{test_name}:")
    print(f"  Accuracy: {metrics['accuracy']:.1f}%")
    print(f"  Total examples: {metrics['total']}")


BASELINE EXPERIMENT (k=3 only) RESULTS

in_distribution:
  Accuracy: 94.8%
  Total examples: 10000

length_gen_k4:
  Accuracy: 0.0%
  Total examples: 1000

length_gen_k5:
  Accuracy: 0.0%
  Total examples: 1000

shift_nines:
  Accuracy: 94.2%
  Total examples: 378

shift_small:
  Accuracy: 92.2%
  Total examples: 1000


In [19]:
# Organize baseline results before moving on
!mkdir -p experiments/baseline_k3_only

# Copy everything from this experiment
!cp -r checkpoints experiments/baseline_k3_only/
!cp results/evaluation_results.json experiments/baseline_k3_only/results.json
!cp data/train.json experiments/baseline_k3_only/train.json
!cp data/val.json experiments/baseline_k3_only/val.json

# Create a quick README
!echo "# Baseline Experiment: k=3 Only Training\n\nTrained on 50K k=3 examples only.\n\n## Files\n- checkpoints/: Model weights\n- results.json: Evaluation metrics\n- train.json: Training data\n- val.json: Validation data" > experiments/baseline_k3_only/README.md

print("✓ Baseline experiment saved to experiments/baseline_k3_only/")
print("\nYou can now safely proceed with mixed k=3,4 training!")

✓ Baseline experiment saved to experiments/baseline_k3_only/

You can now safely proceed with mixed k=3,4 training!


In [20]:
# In Colab: Download just the baseline
from google.colab import files

!zip -r baseline_experiment.zip \
    experiments/baseline_k3_only/ \
    results/evaluation_results.json \
    checkpoints/ \
    *.py \
    config.py \
    requirements.txt

files.download('baseline_experiment.zip')

  adding: experiments/baseline_k3_only/ (stored 0%)
  adding: experiments/baseline_k3_only/checkpoints/ (stored 0%)
  adding: experiments/baseline_k3_only/checkpoints/training_history.json (deflated 65%)
  adding: experiments/baseline_k3_only/checkpoints/.gitkeep (stored 0%)
  adding: experiments/baseline_k3_only/checkpoints/final_model.pt (deflated 9%)
  adding: experiments/baseline_k3_only/checkpoints/best_model.pt (deflated 9%)
  adding: experiments/baseline_k3_only/val.json (deflated 87%)
  adding: experiments/baseline_k3_only/train.json (deflated 87%)
  adding: experiments/baseline_k3_only/README.md (deflated 27%)
  adding: experiments/baseline_k3_only/results.json (deflated 84%)
  adding: results/evaluation_results.json (deflated 84%)
  adding: checkpoints/ (stored 0%)
  adding: checkpoints/training_history.json (deflated 65%)
  adding: checkpoints/.gitkeep (stored 0%)
  adding: checkpoints/final_model.pt (deflated 9%)
  adding: checkpoints/best_model.pt (deflated 9%)
  adding: c

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
# 1. Pull the updated code
# Reset to match GitHub (this will overwrite local Colab changes)
!git fetch origin
!git reset --hard origin/main


HEAD is now at ff3a014 Merge branch 'main' of https://github.com/elenaajayi/learning-addition-transformer


In [27]:
!python data_generator.py


GENERATING ADDITION DATASETS

Configuration:
  k_digits: 3
  Number range: [100, 999]

[Generating training data...]
[Generating validation data...]
[Generating test data...]

[Saving datasets...]
Saved 50000 examples to data/train.json
Saved 5000 examples to data/val.json
Saved 10000 examples to data/test.json

----------------------------------------
DATASET STATISTICS
----------------------------------------

Train set:
  Total examples: 50,000
  With carry: 43,271 (86.5%)
  Without carry: 6,729 (13.5%)

Val set:
  Total examples: 5,000
  With carry: 4,311 (86.2%)
  Without carry: 689 (13.8%)

Test set:
  Total examples: 10,000
  With carry: 8,633 (86.3%)
  Without carry: 1,367 (13.7%)


[Generating length generalization tests...]
Saved 1000 examples to data/test_length_k4.json
Saved 1000 examples to data/test_length_k5.json

[Generating distribution shift tests...]
Saved 378 examples to data/test_shift_many_nines.json
Saved 1000 examples to data/test_shift_small_numbers.json

GENER

In [28]:
!python train.py

ADDITION TRANSFORMER TRAINING

[Setup]
  Device: cuda
  Random seed: 42
  Vocabulary size: 15

[Data]
  Found existing mixed datasets in data/
Loaded 50000 examples from data/train_mixed.json
Loaded 5000 examples from data/val_mixed.json
  Training batches: 391
  Validation batches: 40

[Model]
  Parameters: 4,746,767
  Architecture: 6 layers, d_model=256

[Training Setup]
  Optimizer: AdamW (lr=0.0001, wd=0.01)
  Scheduler: CosineAnnealingLR (T_max=50)
  Loss: CrossEntropyLoss (ignore_index=0)

[Training]
  Max epochs: 50
  Early stopping patience: 5
------------------------------------------------------------

Epoch 1/50
  Train Loss: 2.0848
  Val Loss: 1.8224 | Val Acc: 0.00% | Answer Acc: 0.00%
  Learning Rate: 1.00e-04
  ✓ Saved best model (val_loss: 1.8224)

Epoch 2/50
  Train Loss: 1.8206
  Val Loss: 1.7872 | Val Acc: 0.00% | Answer Acc: 0.00%
  Learning Rate: 9.99e-05
  ✓ Saved best model (val_loss: 1.7872)

Epoch 3/50
  Train Loss: 1.7929
  Val Loss: 1.7519 | Val Acc: 0.00% | 

In [29]:
!python evaluate.py

Loaded model from checkpoints/best_model.pt on cuda

RUNNING FULL EVALUATION


Evaluating: In-Distribution (k=3)
In-Distribution (k=3): 100% 10000/10000 [03:31<00:00, 47.36it/s]

Evaluating: Length Gen (k=4)
Length Gen (k=4): 100% 1000/1000 [00:22<00:00, 43.69it/s]

Evaluating: Length Gen (k=5)
Length Gen (k=5): 100% 1000/1000 [00:12<00:00, 83.31it/s]

Evaluating: Dist Shift (Many 9s)
Dist Shift (Many 9s): 100% 378/378 [00:08<00:00, 46.87it/s]

Evaluating: Dist Shift (Small Nums)
Dist Shift (Small Nums): 100% 1000/1000 [00:18<00:00, 53.43it/s]


┌────────────────────────────┬────────┬──────────┬────────────┬────────────┐
│Test Set                    │ Total  │ Accuracy │ Carry Acc  │  No-Carry  │
├────────────────────────────┼────────┼──────────┼────────────┼────────────┤
│In-Distribution (k=3)       │ 10000  │  87.6%   │   86.5%    │   95.0%    │
│Length Gen (k=4)            │  1000  │  78.9%   │   78.7%    │   80.8%    │
│Length Gen (k=5)            │  1000  │   0.0%   │    0.0%    │

In [30]:
# Create experiment folder
!mkdir -p experiments/mixed_k3_k4


In [31]:
# Copy everything from this training run
!cp -r checkpoints experiments/mixed_k3_k4/
!cp results/evaluation_results.json experiments/mixed_k3_k4/results.json
!cp data/train_mixed.json experiments/mixed_k3_k4/train.json
!cp data/val_mixed.json experiments/mixed_k3_k4/val.json


In [32]:
# Create a README documenting the mixed experiment
readme_content = """# Mixed k=3,4 Training Experiment

## Setup
- Training data: 25,000 k=3 + 25,000 k=4 examples (50K total)
- Validation data: 2,500 k=3 + 2,500 k=4 examples (5K total)
- Epochs: 50
- Final answer accuracy: 82.6%

## Results Summary

| Test Set | Accuracy |
|----------|----------|
| k=3 (in-distribution) | 87.6% |
| k=4 (in-distribution) | 78.9% |
| k=5 (out-of-distribution) | 0.0% |
| Distribution Shift (Many 9s) | 86.8% |
| Distribution Shift (Small Numbers) | 81.6% |

## Key Findings

### Success: Breaking Positional Memorization
Compared to baseline (k=3 only training which achieved 0% on k=4), mixed training
achieved 78.9% on k=4. This demonstrates that variable-length training successfully
broke rigid positional patterns.

### Limitation: Bounded Generalization
Despite success on k=4, the model still achieves 0% on k=5 (never-seen length).
Error analysis shows the model outputs 2-digit answers for 5-digit problems,
suggesting it learned "flexible positions within k=3,4" rather than the general
addition algorithm.

## Files
- `checkpoints/`: Model weights and training history
- `results.json`: Full evaluation metrics
- `train.json`: Training data (25K k=3 + 25K k=4)
- `val.json`: Validation data (2.5K k=3 + 2.5K k=4)

## Comparison to Baseline
See `../baseline_k3_only/` for comparison with k=3-only training.
"""

with open('experiments/mixed_k3_k4/README.md', 'w') as f:
    f.write(readme_content)

print("✓ README created!")

✓ README created!


In [33]:
from google.colab import files

# Zip everything needed for the mixed experiment
!zip -r mixed_k3_k4_final.zip \
    experiments/mixed_k3_k4/ \
    results/evaluation_results.json

print("Downloading mixed_k3_k4_final.zip...")
files.download('mixed_k3_k4_final.zip')

  adding: experiments/mixed_k3_k4/ (stored 0%)
  adding: experiments/mixed_k3_k4/checkpoints/ (stored 0%)
  adding: experiments/mixed_k3_k4/checkpoints/training_history.json (deflated 65%)
  adding: experiments/mixed_k3_k4/checkpoints/.gitkeep (stored 0%)
  adding: experiments/mixed_k3_k4/checkpoints/final_model.pt (deflated 9%)
  adding: experiments/mixed_k3_k4/checkpoints/best_model.pt (deflated 9%)
  adding: experiments/mixed_k3_k4/val.json (deflated 87%)
  adding: experiments/mixed_k3_k4/train.json (deflated 87%)
  adding: experiments/mixed_k3_k4/README.md (deflated 48%)
  adding: experiments/mixed_k3_k4/results.json (deflated 84%)
  adding: results/evaluation_results.json (deflated 84%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>